In [1]:
#!/usr/bin/python
# -*- coding: utf-8 -*-
import urllib.request
import urllib.parse
import urllib.error
from bs4 import BeautifulSoup
import ssl
import json

ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

# url=input("Enter Amazon Product Url- ")
url = 'https://www.amazon.com/Bose-QuietComfort-Bluetooth-Headphones-Noise-Cancelling/dp/B0756GB78C/ref=pd_rhf_ee_s_gcx-rhf_0_2/137-0404929-6941510?_encoding=UTF8&pd_rd_i=B0756GB78C&pd_rd_r=cbeded1e-1e7e-403d-990d-bee197296826&pd_rd_w=LdUMW&pd_rd_wg=HEA5f&pf_rd_p=362afe23-d338-4a7e-9543-e1a204d5662e&pf_rd_r=KVTN44M0EK8MKZASATZA&psc=1&refRID=KVTN44M0EK8MKZASATZA'
html = urllib.request.urlopen(url, context=ctx).read()
soup = BeautifulSoup(html, 'html.parser')
html = soup.prettify('utf-8')
product_json = {}

for divs in soup.findAll('div', attrs={'class': 'a-box-group'}):
    try:
        product_json['brand'] = divs['data-brand']
        break
    except:
        pass

for spans in soup.findAll('span', attrs={'id': 'productTitle'}):
    name_of_product = spans.text.strip()
    product_json['name'] = name_of_product
    break

for divs in soup.findAll('div'):
    try:
        price = str(divs['data-asin-price'])
        product_json['price'] = '$' + price
        break
    except:
        pass

for divs in soup.findAll('div', attrs={'id': 'rwImages_hidden'}):
    for img_tag in divs.findAll('img', attrs={'style': 'display:none;'}):
        product_json['img-url'] = img_tag['src']
        break

for i_tags in soup.findAll('i',
                           attrs={'data-hook': 'average-star-rating'}):
    for spans in i_tags.findAll('span', attrs={'class': 'a-icon-alt'}):
        product_json['star-rating'] = spans.text.strip()
        break

for spans in soup.findAll('span', attrs={'id': 'acrCustomerReviewText'
                          }):
    if spans.text:
        review_count = spans.text.strip()
        product_json['customer-reviews-count'] = review_count
        break

product_json['details'] = []
for ul_tags in soup.findAll('ul',
                            attrs={'class': 'a-unordered-list a-vertical a-spacing-none'
                            }):
    for li_tags in ul_tags.findAll('li'):
        for spans in li_tags.findAll('span',
                attrs={'class': 'a-list-item'}, text=True,
                recursive=False):
            product_json['details'].append(spans.text.strip())

product_json['short-reviews'] = []
for a_tags in soup.findAll('a',
                           attrs={'class': 'a-size-base a-link-normal review-title a-color-base a-text-bold'
                           }):
    short_review = a_tags.text.strip()
    product_json['short-reviews'].append(short_review)

product_json['long-reviews'] = []
for divs in soup.findAll('div', attrs={'data-hook': 'review-collapsed'}):
    long_review = divs.text.strip()
    product_json['long-reviews'].append(long_review)

with open('output_file.html', 'wb') as file:
    file.write(html)
with open('product.json', 'w') as outfile:
    json.dump(product_json, outfile, indent=4)
print('----------Extraction of data is complete. Check json file.----------')

----------Extraction of data is complete. Check json file.----------


In [2]:
import json

In [3]:
json.load(open('product.json').read()).keys()

AttributeError: 'str' object has no attribute 'read'

In [9]:
# Sep 15, 2019, Yingqi Ding, HopHack Fall 2019
from tempfile import mkstemp
from shutil import move
from os import fdopen, remove
import os
import pandas as pd

url = 'https://www.amazon.in/Apple-MacBook-Air-13-3-inch-MQD32HN/product-reviews/B073Q5R6VR/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber='
fh, abs_path = mkstemp()
file_path = 'AutoRun-Spider/amazon_reviews.py'
with fdopen(fh,'w') as new_file:
    with open(file_path) as old_file:
        for line in old_file:
            if 'myBaseUrl = https://' in line:
                new_file.write('    myBaseUrl = ' + url)
            else:
                new_file.write(line)

remove(file_path)
move(abs_path, file_path)

'AutoRun-Spider/amazon_reviews.py'

In [13]:
os.system("scrapy runspider AutoRun-Spider/amazon_reviews.py -o reviews.csv")

0

In [14]:
data = pd.read_csv('reviews.csv')
rating = []
review = []
for i in range(len(data['stars'])):
      if data['stars'][i][:3] != 'sta':
        rating.append(float(data['stars'][i][:3]))
        review.append(data['comment'][i])

In [15]:
review

['I give it five stars. My wife hates them she would give it 1. I put them on and that’s it. I can’t hear her any more.\n            ',
 'I purchased these and the Sony WH1000XM2 to compare the two. Cnet says they both have a "9" for sound quality. I would agree, they both sound excellent. The Bose won the test for its noise cancellation, performance talking to people on the phone, comfort on my head, and sound processing.Phone performance:I compared how the Bose and the Sony sounded when recording and playing back my voice with a fan running in the background. The Bose sounded like I was holding an old fashioned handset and talking in a quiet room - intimate and zero background noise. The Sony sounded like I was on speaker phone, and I could hear some background noise. (As a control I also recorded using neither and it sounded like I was on speaker but also I could hear more background noise.) This feature is important to me since I spend a lot of time on the phone and prefer my clien